In [1]:
import re
import sqlite3
import pandas as pd
from textblob import TextBlob
from collections import Counter

In [2]:
def get_tweet_sentiment(tweet):
    """
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    """

    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet['tweetText']))
    
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [3]:
def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [4]:
# Setup sqlite
sqlite_file = 'eurovision_semi1.db'

# Connect to the database sqlite file
connection = sqlite3.connect(sqlite_file)
db = connection.cursor()

In [113]:
# set country hashtags - semifinal 1
all_hashtags = ['SWE', 'GEO', 'AUS', 'ALB', 'BEL', 'MNE', 'FIN', 'AZE', 'POR',\
                'POL', 'MDA', 'ISL', 'CZE', 'CYP', 'ARM', 'SLO', 'LAT', 'GRE',\
                'AUT', 'BLR', 'DEN', 'EST', 'MKD', 'HUN', 'IRL', 'ISR', 'LTU',\
                'NOR', 'ROM', 'SMR', 'SRB', 'SUI', 'NED', 'CRO', 'BUL', 'MLT',\
                'ITA', 'FRA', 'ESP', 'GBR', 'UKR', 'GER']

hashtags_semi1 = ['SWE', 'GEO', 'AUS', 'ALB', 'BEL', 'MNE', 'FIN', 'AZE', 'POR', 'GRE',\
                    'POL', 'MDA', 'ISL', 'CZE', 'CYP', 'ARM', 'SLO', 'LAT']

# set country hashtags - semifinal 2
hashtags_semi2 = ['AUT', 'BLR', 'DEN', 'EST', 'MKD', 'HUN', 'IRL', 'ISR', 'LTU', 'MLT', \
                    'NOR', 'ROM', 'SMR', 'SRB', 'SUI', 'NED', 'CRO', 'BUL']

# set country hashtags - final
hashtags_final = ['ARM', 'AZE', 'ITA', 'MDA', 'POL', 'POR', 'UKR', 'AUS', 'BEL', 'CYP', 'FRA',\
                  'GER', 'GRE', 'ESP', 'GBR', 'SWE', '???', '???', '???', '???', '???',\
                  '???', '???', '???', '???', '???']

hashtags = hashtags_semi1

In [121]:
# read ALL tweets in english from db, evaluate sentiment, and count
all_sentiments = []
for country in hashtags:

    # get tweets from DB
    country_tweets = pd.read_sql_query("SELECT * FROM TweetsRaw WHERE language='en' AND tweetText LIKE '%#{}%'".format(country), connection)

    # count number of sentiments
    sentiments_count = Counter(country_tweets.apply(get_tweet_sentiment, axis=1))
    
    # append country to list
    all_sentiments.append({'country': country, \
                           'positive': sentiments_count['positive'],\
                           'neutral': sentiments_count['neutral'],\
                           'negative': sentiments_count['negative']
                          })

In [122]:
# read all tweets (to just count)
all_tweet_counts = []
for country in hashtags:

    # get tweet count from DB
    db.execute("SELECT COUNT(*) AS count FROM TweetsRaw WHERE tweetText LIKE '%#{}%'".format(country))
    country_tweet_count = db.fetchone()[0]
    
    # append country to list
    all_tweet_counts.append({'country': country, \
                           'count': country_tweet_count
                          })

In [142]:
# transform to pandas dataframe and add total tweet count
results = pd.DataFrame(all_sentiments)
results = results.set_index(['country'])
results['tweets'] = [tc['count'] for tc in all_tweet_counts]
results['tweets_norm'] = results['tweets'] / results['tweets'].sum()

In [143]:
# add finalist 'column'
results['finalist'] = False
results.loc['MOL','finalist'] = True
results.loc['AZE','finalist'] = True
results.loc['GRE','finalist'] = True
results.loc['SWE','finalist'] = True
results.loc['POR','finalist'] = True
results.loc['POL','finalist'] = True
results.loc['ARM','finalist'] = True
results.loc['AUS','finalist'] = True
results.loc['CYP','finalist'] = True
results.loc['BEL','finalist'] = True

In [148]:
# compute final scores
results['score'] = results['tweets_norm'] * results['positive'] / (results['tweets_norm'] + results['positive'])

In [149]:
# Top-10
print("Accuracy: {}%".format(10*results.sort_values(by = 'score', ascending = False)[0:10]['finalist'].sum()))
results.sort_values(by = 'tweets', ascending = False)[0:10]

Accuracy: 70%


,negative,neutral,positive,tweets,tweets_norm,finalist,score
country,,,,,,,
POR,30.0,177.0,275.0,1265.0,0.262339,True,0.262089
CYP,43.0,198.0,87.0,542.0,0.112401,True,0.112256
BEL,13.0,106.0,93.0,347.0,0.071962,True,0.071906
POL,5.0,77.0,50.0,263.0,0.054542,True,0.054482
LAT,14.0,91.0,48.0,257.0,0.053297,False,0.053238
SWE,14.0,100.0,40.0,239.0,0.049564,True,0.049503
CZE,17.0,65.0,82.0,236.0,0.048942,False,0.048913
FIN,8.0,79.0,57.0,232.0,0.048113,False,0.048072
AUS,21.0,90.0,53.0,224.0,0.046454,True,0.046413
